In [1]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
ollama_api = "http://localhost:11434/api/chat"
headers = {"Content-Type": "application/json"}
Model='llama3.2'

In [3]:
class Website:
    url:str
    title:str
    text=str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup=BeautifulSoup(response.text, "html.parser")
        self.title = soup.title.string if soup.title else "No title"
        for irrelevant in soup(["script", "style","img","input"]):
            irrelevant.decompose()
        self.text = soup.get_text(separator="\n",strip=True)

In [4]:
ed=Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home - Edward Donner
Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage

In [5]:
system_prompt="You are an assistant that analyzes the contents of a website\
and provides a short summary, ignoring text might be navingation related.\
respond in markdown."

In [6]:
def user_prompt_for(website):
    user_prompt=f"You are looking at a website titled {website.title}"
    user_prompt+="\nThe contents of this website are as follows; \
please provide a short summary of the website in markdown. \
if it includes news or announcements, please include that in the summary.\n\n"
    user_prompt+=website.text
    return user_prompt

In [7]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website are as follows; please provide a short summary of the website in markdown. if it includes news or announcements, please include that in the summary.

Home - Edward Donner
Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI sta

In [25]:
def message_for(website):
    return [
        {"role": "system", "content": system_prompt},  # Ollama supports system messages
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [26]:
message_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a websiteand provides a short summary, ignoring text might be navingation related.respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website are as follows; please provide a short summary of the website in markdown. if it includes news or announcements, please include that in the summary.\n\nHome - Edward Donner\nHome\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where i

In [27]:
def summarize(url):
    website=Website(url)
    messages=message_for(website)
    # print(messages)
    payload = {
    "model": Model,
    "messages": messages,
    "stream": False
  }
    response=requests.post(ollama_api, json=payload, headers=headers)
    return response.json()['message']['content']

In [28]:
summarize("https://edwarddonner.com")

"**Summary of the Website**\n\n* **Name**: Home - Edward Donner\n* **Description**: A personal website by Ed, a co-founder and CTO of Nebula.io, an AI startup that applies AI to help people discover their potential.\n* **Content**:\n\t+ A blog about Ed's interests in writing code, experimenting with LLMs, DJing, and amateur electronic music production.\n\t+ Information about his work at Nebula.io, including a patented matching model and award-winning platform.\n\t+ Recent news announcements:\n\t\t- January 23, 2025: LLM Workshop – Hands-on with Agents – resources\n\t\t- December 21, 2024: Welcome, SuperDataScientists!\n\t\t- November 13, 2024: Mastering AI and LLM Engineering – Resources\n\t\t- October 16, 2024: From Software Engineer to AI Data Scientist – resources\n* **Social Media**:\n\t+ LinkedIn\n\t+ Twitter\n\t+ Facebook\n\t+ Newsletter subscription"